In [20]:
import os
import shutil

jsonl_path = "data/dataset_new.jsonl"
save_path = "data/dataset_new"

if os.path.exists(jsonl_path):
    os.remove(jsonl_path)

if os.path.exists(save_path):
    os.remove(save_path)

directory = "data"
if not os.path.exists(directory):
    os.makedirs(directory)


In [8]:
from datasets import load_dataset
import datasets

dic = {
    0: "negtive",
    1: "positive",
    2: "neutral",
}

tfns = load_dataset('zeroshot/twitter-financial-news-sentiment')
tfns = tfns['train']

Using the latest cached version of the dataset since zeroshot/twitter-financial-news-sentiment couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/hsy/.cache/huggingface/datasets/zeroshot___twitter-financial-news-sentiment/default/0.0.0/ccbe24de388e287beb92dd393a335c376b350ac3 (last modified on Thu May  9 17:46:13 2024).


In [10]:
tfns

Dataset({
    features: ['text', 'label'],
    num_rows: 9543
})

In [11]:
tfns = tfns.to_pandas()
tfns

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2
9539,Tupperware Brands among consumer gainers; Unil...,2
9540,vTv Therapeutics leads healthcare gainers; Myo...,2
9541,"WORK, XPO, PYX and AMKR among after hour movers",2


In [12]:
tfns['label'] = tfns['label'].apply(lambda x:dic[x])
tfns

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,negtive
1,$CCL $RCL - Nomura points to bookings weakness...,negtive
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",negtive
3,$ESS: BTIG Research cuts to Neutral https://t....,negtive
4,$FNKO - Funko slides after Piper Jaffray PT cu...,negtive
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,neutral
9539,Tupperware Brands among consumer gainers; Unil...,neutral
9540,vTv Therapeutics leads healthcare gainers; Myo...,neutral
9541,"WORK, XPO, PYX and AMKR among after hour movers",neutral


In [15]:
tfns['instruction'] = 'What is the sentiment of the following tweet? Please choose from the following options: positive, negative, neutral.'
tfns.columns = ['input', 'output', 'instruction']
tfns = datasets.Dataset.from_pandas(tfns)
tfns

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9543
})

In [16]:
tmp_dataset = datasets.concatenate_datasets([tfns]*2)
train_dataset = tmp_dataset

all_dataset = train_dataset.shuffle(seed = 42)
all_dataset.shape

(19086, 3)

In [18]:
import json
from tqdm import tqdm

In [19]:
def save_dataset(dataset, path):
    data_list = []
    for item in dataset.to_pandas().itertuples():
        tmp = {}
        tmp['instruction'] = item.instruction
        tmp['input'] = item.input
        tmp['output'] = item.output
        data_list.append(tmp)

    with open(path, 'w') as f:
        for example in tqdm(data_list, desc='Saving and Formatting dataset'):
            context = f"Instruction: {example['instruction']}\n"
            if example.get('input'):
                context += f"Input: {example['input']}\n"
            context += "Answer: "
            target = example['output']
            f.write(json.dumps({'context': context, 'target': target}) + '\n')
    
save_dataset(all_dataset, jsonl_path)

Saving and Formatting dataset: 100%|██████████| 19086/19086 [00:00<00:00, 160425.82it/s]


Tokenization
convert the input text into tokens that can be fed into the model

In [23]:
import datasets
from transformers import AutoTokenizer, AutoConfig

model_name = "THUDM/chatglm3-6b"
max_seq_length = 512
skip_overlength = True

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    )

config = AutoConfig.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map='auto',
)


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example['context']
    target = example['target']
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target, 
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False,
    )
    input_ids = prompt_ids + target_ids + [tokenizer.eos_token_id]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

In [27]:
jsonl_path = "data/tfns_dataset.jsonl"
def read_jsonl(path, tokenizer, config, max_seq_length, skip_overlength):
    with open(path, "r") as f:
        for line in tqdm(f.readlines(), desc='Preprocessing dataset'):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature

save_path = "data/tfns_dataset"
dataset = datasets.Dataset.from_generator(
    lambda: read_jsonl(jsonl_path, tokenizer, config, max_seq_length, skip_overlength)
)
dataset.save_to_disk(save_path)

Saving the dataset (1/1 shards): 100%|██████████| 19086/19086 [00:00<00:00, 743077.54 examples/s]
